In [1]:
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
from datetime import datetime
import db_multiproc_calc_neutral2 as dmc
from random import *
import barchart_lib as bl

rf_rate = 0.0475
k_do_download = True

cwd = os.getcwd()

executable_path = os.path.join(cwd,'chromedriver.exe')
executable_path = os.path.join(cwd,'chromedriver')
display(executable_path)

date_str = datetime.now().strftime("%Y-%m-%d")
date_str = '03-31-2023'

file_date_str = date_str

download_path = os.path.join(cwd, 'local_download')
file_date_str

'/home/gitrepo/futureDataCapture/chromedriver'

'03-31-2023'

In [2]:
#############################
# get the Futures contracts and months we are needing and build a list of all the distinct contracts

df_future_month_def = pd.read_csv(os.path.join(download_path,'futures_months_config.csv'))

target_years = [23]

all_syms = []
for idx, row in df_future_month_def.iterrows():
    for y in target_years:
        for m in row['Month']:
            all_syms.append(row['symbol_root']+m+f'{y:.0f}')

In [3]:
if k_do_download:
    driver = bl.bar_chart_get_driver(executable_path, download_path)
    bl.bar_chart_login(driver)

In [4]:
contract_specs = []
for sym in all_syms:
    print(sym)
    time.sleep(random()*2)
    contract_specs.append(bl.get_futures_contract_specs(driver, sym))

SIH23
SIK23
SIN23
SIU23
SIZ23
GCG23
GCJ23
GCM23
GCQ23
GCV23
GCZ23
HGH23
HGK23
HGN23
HGU23
HGZ23
NGF23
NGG23
NGH23
NGJ23
NGK23
NGM23
NGN23
NGQ23
NGU23
NGV23
NGX23
NGZ23
CLF23
CLG23
CLH23
CLJ23
CLK23
CLM23
CLN23
CLQ23
CLU23
CLV23
CLX23
CLZ23
ZCH23
ZCK23
ZCN23
ZCU23
ZCZ23
ZSF23
ZSH23
ZSK23
ZSN23
ZSQ23
ZSU23
ZSX23
ZWK23
ZWH23
ZWN23
ZWU23
ZWZ23
ZNH23
ZNM23
ZNU23
ZNZ23
ZFH23
ZFM23
ZFU23
ZFZ23
ZTH23
ZTM23
ZTU23
ZTZ23
SQF23
SQG23
SQH23
SQJ23
SQK23
SQM23
SQN23
SQQ23
SQU23
SQV23


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#main-content-column > div > div.barchart-content-block.commodity-profile > div.block-content > div > div:nth-child(2) > div:nth-child(4) > div.small-7.column.text-right > span > span"}
  (Session info: chrome=111.0.5563.64)
Stacktrace:
#0 0x555b25417243 <unknown>
#1 0x555b251db7a6 <unknown>
#2 0x555b2521864d <unknown>
#3 0x555b25218761 <unknown>
#4 0x555b25253da4 <unknown>
#5 0x555b252390ad <unknown>
#6 0x555b25251932 <unknown>
#7 0x555b25238e53 <unknown>
#8 0x555b2520b9ea <unknown>
#9 0x555b2520cb2e <unknown>
#10 0x555b2546bd5e <unknown>
#11 0x555b2546fa80 <unknown>
#12 0x555b254518b0 <unknown>
#13 0x555b25470b63 <unknown>
#14 0x555b25442f75 <unknown>
#15 0x555b25493998 <unknown>
#16 0x555b25493b27 <unknown>
#17 0x555b254aec23 <unknown>
#18 0x7f831eff1609 start_thread


In [5]:
df_contracts = pd.DataFrame([all_syms, contract_specs]).T

In [6]:
df_contracts.to_csv('future_contract_spec.2.csv')

In [7]:
driver.quit()